In [ ]:
%conda install --file requirements-poetry.txt

: 

In [1]:
# download assets from block storage at given URL 
# (our case is zipped folder served through LAN from another machine by miniserve)
%reload_ext dotenv
# %dotenv
import os
from urllib.request import urlretrieve
import tarfile

ASSET_URL, out_file= (os.environ['ASSET_URL'] ,'out/testing_data.tar.gz')
os.makedirs(os.path.dirname(out_file), exist_ok=True)


folder_out, res = urlretrieve(ASSET_URL, filename=out_file)
print(folder_out)
# subprocess.call(f'wget -P {ASSET_DIR_OUT}/ --recursive --no-parent http://${ASSET_URL}/${ASSET_DIR}', shell=True)

out/testing_data.tar.gz


In [2]:
# Extract files from tar
with tarfile.open(out_file, "r") as file:
    for each in file.getnames():
        print(each)
        file.extract(f'{each}')

data
data/en.openfoodfacts.org.products.csv
data/en.openfoodfacts.org.products-delimited.csv


In [1]:
import polars as pl
# df=pl.read_parquet_schema('./export.parquet')
df=pl.read_parquet('./export.parquet')
# print(df)
df.head(3)

ModuleNotFoundError: No module named 'polars'

In [1]:
import polars as pl
df=pl.read_parquet_schema('./export.parquet')
print(df)


: 

In [8]:
import polars as pl

df = pl.scan_csv("./data/en.openfoodfacts.org.products.csv",
                 truncate_ragged_lines=True,
                 )
df.sink_parquet('./data/openfoodfacts_products.parquet',
                compression="snappy",
                row_group_size=100_00,
                statistics=True,
                )